### NAIP On AWS

This Jupyter notebook provides code to scrape the [NAIP on AWS](https://registry.opendata.aws/naip/) manifest and create a GPKG file that provides a geospatial footprint for all available imagery.

Then, if we are interested in a given OSM ID, it's easy to look up which NAIP images intersect this ID and download them directly.

In [2]:
import pathlib

from tqdm import tqdm
from osgeo import ogr, osr
ogr.UseExceptions()

from rsc.common import aws_naip
from rsc.common.aws_naip import AWS_PATH

# Get NAIP manifest
manifest = aws_naip.get_naip_manifest()

# Filter out shapefiles to download
shp = [e for e in manifest if e.split('.')[-1].lower() in \
    ('shp', 'dbf', 'shx', 'prj', 'sbn')]

# Fetch all the shapefiles
for object_name in tqdm(shp):
    aws_naip.get_naip_file(object_name)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1315/1315 [14:49<00:00,  1.48it/s]


In [3]:
def conv(s: str) -> str:
    """ Convert object paths as seen in manifest to those that might be seen in the shapefiles.
        It's silly they don't match."""
    return '%s.tif' % '_'.join(s.split('_')[:6])

# Read the manifest, and convert the TIF files to those that might be seen in the shapefile metadata
with open(AWS_PATH / 'manifest.txt', 'r') as f:
    mani = {conv(pathlib.Path(p).stem): p for p in (e.strip() for e in f.readlines()) if p.endswith('.tif')}

In [5]:
# Do all the work!

# Create SRS (EPSG:4326: WGS-84 decimal degrees)
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)
srs.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)

# Create GPKG file for writing
driver: ogr.Driver = ogr.GetDriverByName('GPKG')
ds_w: ogr.DataSource = driver.CreateDataSource(str(AWS_PATH / 'naip_on_aws_all_states.gpkg'))
layer_w: ogr.Layer = ds_w.CreateLayer('footprints', srs=srs, geom_type=ogr.wkbPolygon)

# Define output fields
state_field = ogr.FieldDefn('STATE', ogr.OFTString)
band_field = ogr.FieldDefn('BAND', ogr.OFTString)
usgs_id_field = ogr.FieldDefn('USGSID', ogr.OFTString)
src_img_date_field = ogr.FieldDefn('SRCIMGDATE', ogr.OFTString)
filename_field = ogr.FieldDefn('FILENAME', ogr.OFTString)
object_field = ogr.FieldDefn('OBJECT', ogr.OFTString)

# Create output fields in layer
layer_w.CreateField(state_field)
layer_w.CreateField(band_field)
layer_w.CreateField(usgs_id_field)
layer_w.CreateField(src_img_date_field)
layer_w.CreateField(filename_field)
layer_w.CreateField(object_field)

# Get layer feature definition to load in features
feat_defn = layer_w.GetLayerDefn()

# Helper function to safely get field values
def get_field_safe(feat, field_names, fallback=""):
    """
    Safely get a field value, trying multiple possible field names.
    
    Args:
        feat: OGR feature
        field_names: List of field names to try
        fallback: Default value if no field is found
    
    Returns:
        Field value or fallback
    """
    for field_name in field_names:
        try:
            value = feat.GetFieldAsString(field_name)
            if value:  # Only return if we got a non-empty value
                return value
        except RuntimeError:
            continue
    return fallback

# Statistics tracking
total_features = 0
features_processed = 0
features_skipped = 0
state_counts = {}

# Loop through all fetched shapefiles
all_shapefiles = list(AWS_PATH.rglob('*.shp'))
print(f'Processing {len(all_shapefiles)} shapefiles...')

for p in all_shapefiles:
    print(f'\nProcessing: {p.name}')
    
    # Load them in OGR, get layer and spatial reference
    ds_r: ogr.DataSource = ogr.Open(str(p))
    layer_r: ogr.Layer = ds_r.GetLayer()
    srs_r: osr.SpatialReference = layer_r.GetSpatialRef()
    srs_r.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)

    # Debug: Print available fields for first feature (optional)
    if total_features == 0:
        feat_debug = layer_r.GetNextFeature()
        if feat_debug:
            feat_defn_debug = feat_debug.GetDefnRef()
            field_count = feat_defn_debug.GetFieldCount()
            print("Available fields in first feature:")
            for i in range(field_count):
                field_defn = feat_defn_debug.GetFieldDefn(i)
                field_name = field_defn.GetName()
                print(f"  {i}: {field_name}")
            layer_r.ResetReading()  # Reset to beginning

    # Loop through the features in the layer
    for _ in range(layer_r.GetFeatureCount()):
        feat_r: ogr.Feature = layer_r.GetNextFeature()
        total_features += 1

        # Quickly crosscheck with manifest. Skip if not in there
        filename = get_field_safe(feat_r, ['FileName', 'FILENAME'])
        if not filename:
            features_skipped += 1
            continue
            
        filename_conv = conv(filename.split('.')[0])
        if not filename_conv in mani:
            features_skipped += 1
            continue

        # Parse remaining metadata with robust field handling
        state = get_field_safe(feat_r, ['ST', 'QUADST', 'STATE'])
        if not state:
            features_skipped += 1
            continue
            
        # Track state counts
        if state not in state_counts:
            state_counts[state] = 0
        state_counts[state] += 1
        
        # Get band information with robust field handling
        band = get_field_safe(feat_r, ['Band', 'BAND'])
        if not band:
            features_skipped += 1
            continue
        
        usgs_id = get_field_safe(feat_r, ['USGSID', 'USGS_ID'])
        src_img_date = get_field_safe(feat_r, ['SrcImgDate', 'SRCIMGDATE', 'DATE'])

        # Fetch geometry and convert to desired spatial reference
        try:
            trans = osr.CoordinateTransformation(srs_r, srs)
            geom = ogr.CreateGeometryFromWkt(feat_r.GetGeometryRef().ExportToWkt())
            geom.Transform(trans)
        except Exception as e:
            print(f"Geometry error in {p.name}: {e}")
            features_skipped += 1
            continue

        # Create our new feature
        feat_w = ogr.Feature(feat_defn)
        feat_w.SetGeometry(geom)
        feat_w.SetField('STATE', state)
        feat_w.SetField('BAND', band)
        feat_w.SetField('USGSID', usgs_id)
        feat_w.SetField('SRCIMGDATE', src_img_date)
        feat_w.SetField('FILENAME', filename)
        feat_w.SetField('OBJECT', mani[filename_conv])

        # Save!
        layer_w.CreateFeature(feat_w)
        features_processed += 1

        # Cleanup features
        feat_w = None
        feat_r = None

    # Cleanup read dataset
    layer_r = None
    ds_r = None

# Cleanup write dataset
layer_w = None
ds_w = None

# Print summary statistics
print(f'\nProcessing complete!')
print(f'Total features examined: {total_features}')
print(f'Features processed: {features_processed}')
print(f'Features skipped: {features_skipped}')
print(f'Success rate: {features_processed/total_features*100:.1f}%')
print(f'\nFeatures by state:')
for state, count in sorted(state_counts.items()):
    print(f'  {state}: {count}')
print(f'\nOutput saved to: {AWS_PATH / "naip_on_aws_all_states.gpkg"}')

Processing 263 shapefiles...

Processing: naip_nm_2011_1m_m4b.shp
Available fields in first feature:
  0: AREA
  1: PERIMETER
  2: ST
  3: QQNAME
  4: QKEY
  5: QUADRANT
  6: APFONAME
  7: GNIS
  8: DY
  9: MY
  10: SY
  11: DX
  12: MX
  13: SX
  14: OLAT
  15: OLONG
  16: ArcKey
  17: Band
  18: USGSID
  19: Qdrnt
  20: UTM
  21: Res
  22: SrcImgDate
  23: VerDate
  24: FileName

Processing: naip_3_14_1_2_nm.shp

Processing: naip_3_16_1_2_nm.shp

Processing: NAIP_20_NM.shp

Processing: NAIP_18_NM.shp

Processing: naip_de_2011_1m_m4b.shp

Processing: DE_NAIP21_QQ.shp

Processing: naip_3_13_2_7_de.shp

Processing: NAIP_18_DE.shp

Processing: naip_3_17_2_5_de.shp

Processing: naip_3_15_2_8_de.shp

Processing: naip_oh_2011_1m_m4b.shp

Processing: OH_NAIP21_QQ.shp

Processing: NAIP_19_OH.shp

Processing: naip_3_13_3_8_oh.shp

Processing: naip_3_17_2_1_oh.shp

Processing: naip_3_15_2_4_oh.shp

Processing: naip_ia_2011_1m_m4b.shp

Processing: naip_3_14_3_2_ia.shp

Processing: IA_NAIP21_QQ.s